In [12]:
import win32com.client
from datetime import datetime
import pytz
import os


def find_folders(outlook, folder_name):
    """
    Recherche récursive d'un dossier Outlook (par nom) dans toutes les boîtes.
    Affiche les dossiers sous forme d'arborescence.
    Retourne une liste de dossiers trouvés.
    """
    found_folders = []

    def search(folder, level=0):
        try:
            indent = "    " * level  # indentation visuelle
            print(f"{indent}📂 {folder.Name}")

            if folder.Name.lower() == folder_name.lower():
                print(f"{indent}✅ Dossier trouvé : {folder.FolderPath}")
                found_folders.append(folder)

            for subfolder in folder.Folders:
                search(subfolder, level + 1)

        except Exception as e:
            print(f"⚠️ Erreur accès dossier {folder.Name}: {e}")

    for store in outlook.Folders:
        print(f"\n=== Boîte : {store.Name} ===")
        search(store)

    return found_folders


def search_outlook_mails(folder_name, sender=None, subject=None,
                         date_after=None, date_before=None, attachments_dir=None):
    outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
    results = []

    # Recherche des dossiers
    target_folders = find_folders(outlook, folder_name)

    if not target_folders:
        print(f"⚠️ Aucun dossier nommé '{folder_name}' trouvé.")
        return []

    for folder in target_folders:
        try:
            print(f"\n📌 Analyse du dossier : {folder.FolderPath}")

            if folder.DefaultItemType != 0:  # 0 = MailItem
                continue

            items = folder.Items
            restrictions = []

            if date_after:
                restrictions.append(f"[ReceivedTime] >= '{date_after.strftime('%m/%d/%Y %I:%M %p')}'")
            if date_before:
                restrictions.append(f"[ReceivedTime] <= '{date_before.strftime('%m/%d/%Y %I:%M %p')}'")

            restriction_query = " AND ".join(restrictions)

            if restriction_query:
                items = items.Restrict(restriction_query)

            items.Sort("[ReceivedTime]", True)

            for item in items:
                if item.Class == 43:
                    match = True

                    if sender and sender.lower() not in (item.SenderEmailAddress or "").lower():
                        match = False
                    if subject and subject.lower() not in (item.Subject or "").lower():
                        match = False

                    if match:
                        mail_info = {
                            "Sender": item.SenderName,
                            "Email": item.SenderEmailAddress,
                            "Subject": item.Subject,
                            "Date": item.ReceivedTime.strftime("%Y-%m-%d %H:%M:%S"),
                            "Folder": folder.FolderPath
                        }

                        # Sauvegarde des pièces jointes
                        if attachments_dir:
                            os.makedirs(attachments_dir, exist_ok=True)
                            for attachment in item.Attachments:
                                try:
                                    save_path = os.path.join(attachments_dir, attachment.FileName)
                                    attachment.SaveAsFile(save_path)
                                    print(f"📎 Pièce jointe sauvegardée : {save_path}")
                                except Exception as e:
                                    print(f"⚠️ Erreur sauvegarde PJ : {e}")

                        results.append(mail_info)

        except Exception as e:
            print(f"⚠️ Erreur dans le dossier {folder.FolderPath}: {e}")

    return results


if __name__ == "__main__":
    tz = pytz.timezone("UTC")
    date_after = tz.localize(datetime(2025, 1, 1))
    date_before = tz.localize(datetime(2025, 8, 1))

    mails = search_outlook_mails(
        folder_name="extractions KYC",                   # 🔹 Nom du dossier recherché
        sender="compliance_eme@gos.orange.com",
        subject="[VITIB][EME][Production][BF01]: Liste alertes kyc ",
        date_after=date_after,
        date_before=date_before,
        attachments_dir=r"C:\Users\Public\PJ"
    )

    print(f"\n--- {len(mails)} mails trouvés ---")
    for mail in mails:
        print(f"[{mail['Date']}] {mail['Sender']} ({mail['Email']}) - {mail['Subject']} (Dossier: {mail['Folder']})")



=== Boîte : Archives_2024 ===
📂 Archives_2024
    📂 Éléments supprimés
    📂 Boîte de réception
    📂 Boîte d'envoi
    📂 Éléments envoyés
    📂 Journal
    📂 Courrier indésirable
    📂 Notes
    📂 Conversation Action Settings
    📂 Calendrier
    📂 Brouillons
    📂 Tâches
    📂 Problèmes de synchronisation
        📂 Conflits
    📂 Flux RSS
    📂 Working Set
    📂 Paramètres des étapes rapides
    📂 Flux d’informations
    📂 Files
    📂 Yammer Root
        📂 Inbound
        📂 Outbound
        📂 Feeds

=== Boîte : Recrutement Orange Money Groupe ===
📂 Recrutement Orange Money Groupe
    📂 ARCHIVES 1
    📂 Tâches
    📂 Racine de Yammer
        📂 Outbound
        📂 Inbound
        📂 Feeds
    📂 Notes
    📂 Journal
    📂 Fichiers
    📂 ExternalContacts
    📂 Éléments supprimés
    📂 Éléments envoyés
    📂 Courrier indésirable
    📂 Conversation Action Settings
    📂 Contacts
        📂 Sociétés
        📂 Organizational Contacts
        📂 PeopleCentricConversation Buddies
        📂 {A9E2BC4